In [ ]:
import geopandas as gpd
import pandas as pd

import folium
from unidecode import unidecode

In [ ]:
df = pd.read_csv('RESULTADOS_2024_CSV_V1.csv')

In [ ]:
# remove since they're already aggregated in 'Otros'
df = df.drop(['Martinez', 'Bertucci', 'Brito', 'Ecarri',
       'Fermin', 'Ceballos', 'Marquez', 'Rausseo', 'votantes'], axis=1)

In [ ]:
df = df.rename(columns={"ESTADO": "Estado", "municipio": "Municipio", "parroquia": "Parroquia"})

In [ ]:
# remove special characters like ~, ´, ¨, and whatnot...
df['Estado'] = df['Estado'].apply(unidecode).str.upper()
df['Municipio'] = df['Municipio'].apply(unidecode).str.upper()
df['Parroquia'] = df['Parroquia'].apply(unidecode).str.upper()

In [ ]:
# change some municipality names so it matches the map data
df.loc[df['Estado'] == 'CAPITAL', 'Estado'] = 'DISTRITO CAPITAL'
df.loc[(df['Estado'] == 'BOLIVAR') & (df['Municipio'] == 'ANGOSTURA'), 'Municipio'] = 'BOLIVARIANO ANGOSTURA'
df.loc[(df['Estado'] == 'ANZOATEGUI') & (df['Municipio'] == 'FREITES'), 'Municipio'] = 'PEDRO MARIA FREITES'
df.loc[(df['Estado'] == 'ANZOATEGUI') & (df['Municipio'] == 'GUANIPA'), 'Municipio'] = 'SAN JOSE DE GUANIPA'
df.loc[(df['Estado'] == 'ANZOATEGUI') & (df['Municipio'] == 'SOTILLO'), 'Municipio'] = 'JUAN ANTONIO SOTILLO'
df.loc[(df['Estado'] == 'ANZOATEGUI') & (df['Municipio'] == 'MONAGAS'), 'Municipio'] = 'JOSE GREGORIO MONAGAS'
df.loc[(df['Estado'] == 'ANZOATEGUI') & (df['Municipio'] == 'PENALVER'), 'Municipio'] = 'FERNANDO DE PENALVER'
df.loc[(df['Estado'] == 'ANZOATEGUI') & (df['Municipio'] == 'BRUZUAL'), 'Municipio'] = 'MANUEL EZEQUIEL BRUZUAL'
df.loc[(df['Estado'] == 'ANZOATEGUI') & (df['Municipio'] == 'CAJIGAL'), 'Municipio'] = 'JUAN MANUEL CAJIGAL'
df.loc[(df['Estado'] == 'ANZOATEGUI') & (df['Municipio'] == 'CARVAJAL'), 'Municipio'] = 'FRANCISCO DEL CARMEN CARVAJAL'
df.loc[(df['Estado'] == 'ANZOATEGUI') & (df['Municipio'] == 'MC GREGOR'), 'Municipio'] = 'SIR ARTHUR MC GREGOR'
df.loc[(df['Estado'] == 'NUEVA ESPARTA') & (df['Municipio'] == 'PENINSULA DE MACANAO'), 'Municipio'] = 'MACANAO'
df.loc[(df['Estado'] == 'GUARICO') & (df['Municipio'] == 'ZARAZA'), 'Municipio'] = 'PEDRO ZARAZA'
df.loc[(df['Estado'] == 'GUARICO') & (df['Municipio'] == 'MONAGAS'), 'Municipio'] = 'JOSE TADEO MONAGAS'
df.loc[(df['Estado'] == 'GUARICO') & (df['Municipio'] == 'INFANTE'), 'Municipio'] = 'LEONARDO INFANTE'
df.loc[(df['Estado'] == 'GUARICO') & (df['Municipio'] == 'MELLADO'), 'Municipio'] = 'JULIAN MELLADO'
df.loc[(df['Estado'] == 'GUARICO') & (df['Municipio'] == 'ROSCIO'), 'Municipio'] = 'JUAN GERMAN ROSCIO'
df.loc[(df['Estado'] == 'MIRANDA') & (df['Municipio'] == 'GUAICAIPURO'), 'Municipio'] = 'BOLIVARIANO GUAICAIPURO'
df.loc[(df['Estado'] == 'ZULIA') & (df['Municipio'] == 'GUAJIRA'), 'Municipio'] = 'INDIGENA BOLIVARIANO GUAJIRA'

In [ ]:
df.loc[df['Estado'] == 'ANZOATEGUI', 'Municipio'].unique()

In [ ]:
df.head()

In [ ]:
# aggregate results by state
df_by_state = df.groupby(['Estado'], as_index=False)[['egu', 'nm', 'otros']].sum()
df_by_state['total'] = df_by_state[['egu', 'nm', 'otros']].sum(axis=1)

# aggregate results by state and municipality
df_by_municipio = df.groupby(['Estado', 'Municipio'], as_index=False)[['egu', 'nm', 'otros']].sum()
df_by_municipio['total'] = df_by_municipio[['egu', 'nm', 'otros']].sum(axis=1)

In [ ]:
# creating a state indexed version of the dataframes so we can lookup values
state_data_indexed = df_by_state.set_index('Estado')

# create a multi index for state - municipality
state_mun_tuples = list(zip(df_by_municipio.Estado, df_by_municipio.Municipio))
index = pd.MultiIndex.from_tuples(state_mun_tuples, names=['Estado', 'Municipio'])
mun_data_indexed = df_by_municipio.set_index(index)

In [ ]:
# load geo data from shape file (level 1 state level)
ve_map = gpd.read_file('ven_adm_ine_20210223_shp/ven_admbnda_adm1_ine_20210223.shp')
ve_map = ve_map.rename(columns={"ADM1_ES": "Estado"})
ve_map['Estado'] = ve_map['Estado'].apply(unidecode).str.upper()

# load geo data from shape file (level 2 state and municipality level)
ve_map_mun = gpd.read_file('ven_adm_ine_20210223_shp/ven_admbnda_adm2_ine_20210223.shp')
ve_map_mun = ve_map_mun.rename(columns={"ADM2_ES": "Municipio", "ADM1_ES": "Estado"})
ve_map_mun['Estado'] = ve_map_mun['Estado'].apply(unidecode).str.upper()
ve_map_mun['Municipio'] = ve_map_mun['Municipio'].apply(unidecode).str.upper()

In [ ]:
ve_map_mun.loc[ve_map_mun['Estado'] == 'GUARICO', 'Municipio'].unique()

In [ ]:
# fix some municipality names
ve_map_mun.loc[(ve_map_mun['Estado'] == 'MONAGAS') & (ve_map_mun['Municipio'] == 'SANTA BABARA'), 'Municipio'] = 'SANTA BARBARA'
ve_map_mun.loc[(ve_map_mun['Estado'] == 'AMAZONAS') & (ve_map_mun['Municipio'] == 'AUTONOMO ATURES'), 'Municipio'] = 'ATURES'
ve_map_mun.loc[(ve_map_mun['Estado'] == 'ARAGUA') & (ve_map_mun['Municipio'] == 'MARIO BRICENO IRAGORRI'), 'Municipio'] = 'MARIO BRICENO IRAGORRY'
ve_map_mun.loc[(ve_map_mun['Estado'] == 'PORTUGUESA') & (ve_map_mun['Municipio'] == 'MONSENOR JOSE VICENTE DE UND'), 'Municipio'] = 'MONSENOR JOSE VICENTE DE UNDA'
ve_map_mun.loc[(ve_map_mun['Estado'] == 'TRUJILLO') & (ve_map_mun['Municipio'] == 'JOSE FELIPE MARQUEZ CANIZAL'), 'Municipio'] = 'JOSE FELIPE MARQUEZ CANIZALES'
ve_map_mun.loc[(ve_map_mun['Estado'] == 'MERIDA') & (ve_map_mun['Municipio'] == 'JULIO CESAR SALA'), 'Municipio'] = 'JULIO CESAR SALAS'
ve_map_mun.loc[(ve_map_mun['Estado'] == 'ZULIA') & (ve_map_mun['Municipio'] == 'JESUS MARIA SEMPRUM'), 'Municipio'] = 'JESUS MARIA SEMPRUN'

In [ ]:
ve_map_mun.loc[ve_map_mun['Estado'] == 'MONAGAS', 'Municipio'].unique()

In [ ]:
# remove date columns
ve_map = ve_map.drop(['date', 'validOn', 'validTo'], axis=1)
ve_map_mun = ve_map_mun.drop(['date', 'validOn', 'validTo'], axis=1)

In [ ]:
# create the map
m = folium.Map(location=[7, -66], zoom_start=6, tiles='cartodbpositron')

In [ ]:
# by state

cp = folium.Choropleth(
    geo_data=ve_map,
    name='Votación por estado',
    data=df_by_state,
    columns=['Estado', 'total'],
    key_on="feature.properties.Estado",
    fill_opacity=0.8,
    line_opacity=0.9,
    legend_name="Votos por estado, 2024",
    highlight=True,
    control=False
).add_to(m)

for s in cp.geojson.data['features']:
    try:
        s['properties']['egu'] = f"{state_data_indexed.loc[s['properties']['Estado'], 'egu'].item():,}"
        s['properties']['nm'] = f"{state_data_indexed.loc[s['properties']['Estado'], 'nm'].item():,}"
        s['properties']['otros'] = f"{state_data_indexed.loc[s['properties']['Estado'], 'otros'].item():,}"
        s['properties']['total'] = f"{state_data_indexed.loc[s['properties']['Estado'], 'total'].item():,}"
        
    except KeyError:
        pass

# and finally adding a tooltip/hover to the choropleth's geojson
folium.GeoJsonTooltip(
    fields=['Estado', 'egu', 'nm', 'otros', 'total'],
    aliases=['Estado', 'Edmundo González', 'Nicolás Maduro', 'Otros', 'Total']).add_to(cp.geojson)

folium.LayerControl().add_to(m)

In [ ]:
m

In [ ]:
m.save('dist/2024_vzla_by_state.html')

In [ ]:
m = folium.Map(location=[7, -66], zoom_start=6, tiles='cartodbpositron')

In [ ]:
# by municipality

cp = folium.Choropleth(
    geo_data=ve_map_mun,
    name='Votación por municipio',
    data=df_by_municipio,
    columns=['Municipio', 'total'],
    key_on="feature.properties.Municipio",
    fill_opacity=0.8,
    line_opacity=0.9,
    legend_name="Votos por municipio, 2024",
    highlight=True,
    control=False
).add_to(m)

for s in cp.geojson.data['features']:
    try:
        s['properties']['egu'] = f"{mun_data_indexed.loc[[(s['properties']['Estado'],s['properties']['Municipio'])], 'egu'].item():,}"
        s['properties']['nm'] = f"{mun_data_indexed.loc[[(s['properties']['Estado'],s['properties']['Municipio'])], 'nm'].item():,}"
        s['properties']['otros'] = f"{mun_data_indexed.loc[[(s['properties']['Estado'],s['properties']['Municipio'])], 'otros'].item():,}"
        s['properties']['total'] = f"{mun_data_indexed.loc[[(s['properties']['Estado'],s['properties']['Municipio'])], 'total'].item():,}"
    except KeyError:
        pass

# and finally adding a tooltip/hover to the choropleth's geojson
folium.GeoJsonTooltip(
    fields=['Estado', 'Municipio', 'egu', 'nm', 'otros', 'total'],
    aliases=['Estado', 'Municipio', 'Edmundo González', 'Nicolás Maduro', 'Otros', 'Total']).add_to(cp.geojson)

folium.LayerControl().add_to(m)

In [ ]:
m

In [ ]:
m.save('dist/2024_vzla_by_municipality.html')